*Copyright (C) 2021 Intel Corporation*<br>
*SPDX-License-Identifier: BSD-3-Clause*<br>
*See: https://spdx.org/licenses/*

---

# Connect processes

This tutorial gives an introduction in how to connect _Processes_ to build a network of asynchronously operating and interacting _Processes_.

## Recommended tutorials before starting:

- [Installing Lava](./tutorial01_installing_lava.ipynb "Tutorial on Installing Lava")
- [Processes](./tutorial02_processes.ipynb "Tutorial on Processes")
- [ProcessModel](./tutorial03_process_models.ipynb "Tutorial on ProcessModels")
- [Execution](./tutorial04_execution.ipynb "Tutorial on Executing Processes")

    
## Building a network of _Processes_

_Processes_ are the main building blocks of Lava. Each _Process_ can exercise different computations and usually depends on some input data and/or creates output data. Transfering I/O data between _Processes_ is a key element of Lava. A _Process_ can have various input and output _Ports_ which are then connected via channels to corresponding _Ports_ of another _Process_. This allows to build networks of asynchronously operating and interacting _Processes_.

## Create a connection

The objective is to connect _Process_ _P1_ with _Process_ _P2_. _P1_ has an output _Port_ _OutPort_ called _out_ and _P2_ has an input port _InPort_ called _inp_. Data from _P1_ provided to the _Port_ _out_ should be transfered to _P2_ and received from _Port_ _inp_.

<img src="https://raw.githubusercontent.com/lava-nc/lava-nc.github.io/main/_static/images/tutorial06/fig01.png" width="400"  />


In [1]:
from lava.magma.core.process.process import AbstractProcess
from lava.magma.core.process.ports.ports import InPort, OutPort

As first step we define the _Processes_ _P1_ and _P2_ with their respective _Ports_ _out_ and _inp_.

In [2]:
# Minimal process with an OutPort
class P1(AbstractProcess):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        shape = kwargs.get('shape', (2,))
        self.out = OutPort(shape=shape)


# Minimal process with an InPort
class P2(AbstractProcess):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        shape = kwargs.get('shape', (2,))
        self.inp = InPort(shape=shape)

_Process_ _P1_ and _P2_ require a corresponding _ProcessModel_ which implements their _Ports_ and a simple RunConfig for sending and receiving data.

The _ProcessModels_ can be written in Python and should be exectued on a CPU. The input and output _Port_ should be able to receive/send a vector of integers and print the transferred data.

So the _ProcessModel_ inherits from _AbstractPyProcessModel_ in order to execute Python code and the configured _ComputeResource_ is a CPU. A _LavaPyType_ is used for the _Ports_. The _LavaPyType_ specifies the expected data format for the _Port_. A dense vector of type integer is chosen with the parameters _PyOutPort._VEC_DENSE and _int_. The _Ports_ can be used to send and receive data by calling _send_ or _recv_. The sent and received data is afterwards printed out.

In [3]:
import numpy as np
from lava.magma.core.model.py.model import PyLoihiProcessModel
from lava.magma.core.decorator import implements, requires, tag
from lava.magma.core.resources import CPU
from lava.magma.core.model.py.type import LavaPyType
from lava.magma.core.model.py.ports import PyInPort, PyOutPort
from lava.magma.core.sync.protocols.loihi_protocol import LoihiProtocol

In [4]:
# A minimal PyProcModel implementing P1
@implements(proc=P1, protocol=LoihiProtocol)
@requires(CPU)
@tag('floating_pt')
class PyProcModelA(PyLoihiProcessModel):
    out: PyOutPort = LavaPyType(PyOutPort.VEC_DENSE, int)

    def run_spk(self):
        data = np.array([1, 2])
        self.out.send(data)
        print("Sent output data of P1: {}".format(data))



# A minimal PyProcModel implementing P2
@implements(proc=P2, protocol=LoihiProtocol)
@requires(CPU)
@tag('floating_pt')
class PyProcModelB(PyLoihiProcessModel):
    inp: PyInPort = LavaPyType(PyInPort.VEC_DENSE, int)

    def run_spk(self):
        in_data = self.inp.recv()
        print("Received input data for P2: {}".format(in_data))

Next, the processes _P1_ and _P2_ are instantiated and the output _Port_ _out_ from _Process_ _P1_ is connected with the input _Port_ _inp_ of _Process_ _P2_.

In [5]:
sender = P1()
recv = P2()

# Connecting output port to an input port
sender.out.connect(recv.inp)

sender = P1()
recv = P2()

# ... or connecting an input port from an output port
recv.inp.connect_from(sender.out)

Calling `run()` on either of these _Processes_ will first call the _Compiler_. During compilation the specified connection is setup by creating a channel between _P1_ and _P2_. Now data can be transfered during execution as seen by the output print statements.

In [6]:
from lava.magma.core.run_configs import Loihi1SimCfg
from lava.magma.core.run_conditions import RunSteps

In [7]:
sender.run(RunSteps(num_steps=1), Loihi1SimCfg())
sender.stop()

Sent output data of P1: [1 2]
Received input data for P2: [1 2]


The instance `sender` of P1 sent the data `[1 2]` via its _OutPort_ `out` to the _InPort_ `in` of the instance `recv` of P2, where the data is received.

## Possible connections
This first example was very simple. In principle, _Processes_ can have multiple input and output _Ports_ which can be freely connected with each other. Also, _Processes_ which execute on different compute resources can be connected in the same way.

<img src="https://raw.githubusercontent.com/lava-nc/lava-nc.github.io/main/_static/images/tutorial06/fig02.png" width="400"  />

#### There are some things to consider though:
- _InPorts_ cannot connect to _OutPorts_
- Shape and datatype of connect _Ports_ must match
- An _InPort_ might get data from multiple _OutPorts_ - default behavior is a summation of the incoming data
- An _OutPort_ might send data to multiple _InPorts_ - all _InPorts_ receive the same data


## Connect multiple _InPorts_ from a single _OutPort_

<img src="https://raw.githubusercontent.com/lava-nc/lava-nc.github.io/main/_static/images/tutorial06/fig04.png" width="400"  />

In [8]:
sender = P1()
recv1 = P2()
recv2 = P2()
recv3 = P2()

# An OutPort can connect to multiple InPorts
# Either at once...
sender.out.connect([recv1.inp, recv2.inp, recv3.inp])

sender = P1()
recv1 = P2()
recv2 = P2()
recv3 = P2()

# ... or consecutively
sender.out.connect(recv1.inp)
sender.out.connect(recv2.inp)
sender.out.connect(recv3.inp)

In [9]:
sender.run(RunSteps(num_steps=1), Loihi1SimCfg())
sender.stop()

Received input data for P2: [1 2]
Sent output data of P1: [1 2]
Received input data for P2: [1 2]
Received input data for P2: [1 2]


The instance `sender` of P1 sent the data `[1 2]` to the 3 instances `recv1, recv2, recv3` of P2.

## Connecting multiple _InPorts_ to a single _OutPort_

If multiple input _Ports_ connect to the same output _Port_ the default behavior is that the data from each input _Port_ is added up at the output _Port_.

<img src="https://raw.githubusercontent.com/lava-nc/lava-nc.github.io/main/_static/images/tutorial06/fig05.png" width="400"  />

In [10]:
sender1 = P1()
sender2 = P1()
sender3 = P1()
recv = P2()

# An InPort can connect to multiple OutPorts
# Either at once...
recv.inp.connect_from([sender1.out, sender2.out, sender3.out])

sender1 = P1()
sender2 = P1()
sender3 = P1()
recv = P2()

# ... or consecutively
sender1.out.connect(recv.inp)
sender2.out.connect(recv.inp)
sender3.out.connect(recv.inp)

In [11]:
sender1.run(RunSteps(num_steps=1), Loihi1SimCfg())
sender1.stop()

Sent output data of P1: [1 2]
Received input data for P2: [3 6]
Sent output data of P1: [1 2]
Sent output data of P1: [1 2]


The 3 instances `sender1, sender2, sender3` of P1 sent the data `[1 2]` to the instance `recv` of P2, where the data was summed up to `[3 6]`.

## How to learn more?

Learn how to implement and compose the behavior of a process using other processes in the [next tutorial on hierarchical Processes](./tutorial06_hierarchical_processes.ipynb "Tutorial on Hierarchical Processes").

If you want to find out more about connecting processes, have a look at the [Lava documentation](https://lava-nc.org/ "Lava Documentation") or dive into the [source code](https://github.com/lava-nc/lava/tree/main/src/lava/magma/core/process/ports/ports.py "Port Source Code").

To receive regular updates on the latest developments and releases of the Lava Software Framework please subscribe to the [INRC newsletter](http://eepurl.com/hJCyhb "INRC Newsletter").